## Hazard indicators
Physrisk is primarily designed to run 'bottom-up' calculations that model the impact of climate hazards on large numbers of individual assets (including natural) and operations. These calculations can be used to assess financial risks or socio-economic impacts. To do this physrisk collects:

- hazard indicators and
- models of vulnerability of assets/operations to hazards.

Hazard indicators – that is, quantities that provide the information about hazards needed by the vulnerability models – are collected from a variety of sources. OS-Climate consolidates public domain hazard indicators and also provides the means to combine these with commercial data.

We start with public domain indicators: which ones are available and how can these be obtained?

#### Requesting hazard indicators via the physrisk API

Using physrisk, hazard indicators can be obtained directly from source, as long as the user has the necessary API keys. Alternatively, the API – which is simply a hosted instance of physrisk – can be used, albeit the number of requests will be restricted to 30,000 latitudes and longitudes.

We give a walk-through of the hazard indicator API below, as well as examples of how to obtain the inventory of available hazard indicators.

In [ ]:
# UNCOMMENT TO RUN THIS NOTEBOOK
# pip install nbformat pandas plotly requests

# UNCOMMENT TO RUN THE SECOND PART OF THIS NOTEBOOK:
# pip install physrisk-lib

import plotly.graph_objs as go
import pprint as pp
import requests

from dotenv import load_dotenv
from IPython.display import Markdown, display
from physrisk.container import Container
from plotly.subplots import make_subplots

In [2]:
base_url = "https://physrisk-api2-sandbox.apps.odh-cl1.apps.os-climate.org/api/"

request = {
    "items": [
        {
            "longitudes": [69.4787, 68.71, 20.1047, 19.8936, 19.6359, 0.5407, 6.9366, 6.935, 13.7319, 13.7319],
            "latitudes": [34.556, 35.9416, 39.9116, 41.6796, 42.0137, 35.7835, 36.8789, 36.88, -12.4706, -12.4706],
            "request_item_id": "my_flood_request",
            "hazard_type": "RiverineInundation",
            "indicator_id": "flood_depth",
            "scenario": "historical",
            "year": 1980,
        },
        {
            "longitudes": [69.4787, 68.71, 20.1047, 19.8936, 19.6359, 0.5407, 6.9366, 6.935, 13.7319, 13.7319],
            "latitudes": [34.556, 35.9416, 39.9116, 41.6796, 42.0137, 35.7835, 36.8789, 36.88, -12.4706, -12.4706],
            "request_item_id": "my_flood_request",
            "hazard_type": "RiverineInundation",
            "indicator_id": "flood_depth",
            "scenario": "rcp8p5",
            "indicator_model_gcm": "NorESM1-M",  # optional: can specify
            "year": 2050,
        },
        {
            "longitudes": [114.089],
            "latitudes": [22.4781],
            "request_item_id": "my_wind_request_ssp585",
            "hazard_type": "Wind",
            "indicator_id": "max_speed",
            "scenario": "historical",
            "path": "wind/iris/v1/max_speed_{scenario}_{year}",
            # if path is specified then that particular data array is used
            "year": 2010,
        },
        {
            "longitudes": [114.089],
            "latitudes": [22.4781],
            "request_item_id": "my_wind_request_histo",
            "hazard_type": "Wind",
            "indicator_id": "max_speed",
            "scenario": "ssp585",
            "path": "wind/iris/v1/max_speed_{scenario}_{year}",
            "year": 2050,
        },
        {
            "longitudes": [114.089],
            "latitudes": [22.4781],
            "request_item_id": "my_fire_request",
            "hazard_type": "Fire",
            "indicator_id": "fire_probability",
            "scenario": "ssp585",
            "path": "fire/jupiter/v1/fire_probability_{scenario}_{year}",
            "year": 2040,
        },
    ]
}
url = base_url + "get_hazard_data"
response = requests.post(url, json=request).json()
flood_results_baseline, flood_results_rcp585 = (
    response["items"][0]["intensity_curve_set"],
    response["items"][1]["intensity_curve_set"],
)
wind_results_baseline, wind_results_ssp585 = (
    response["items"][2]["intensity_curve_set"],
    response["items"][3]["intensity_curve_set"],
)
fire_results = response["items"][4]["intensity_curve_set"]

In [4]:
fig1 = make_subplots(rows=1, cols=2)
fig1.add_trace(
    go.Scatter(
        x=flood_results_baseline[0]["index_values"], y=flood_results_baseline[0]["intensities"], name="baseline flood"
    ),
    row=1,
    col=1,
)
fig1.add_trace(
    go.Scatter(
        x=flood_results_rcp585[0]["index_values"], y=flood_results_rcp585[0]["intensities"], name="flood RCP 8.5 2050"
    ),
    row=1,
    col=1,
)
fig1.update_xaxes(title="Return period (years)", title_font={"size": 14}, row=1, col=1, type="log")
fig1.update_yaxes(title="Flood depth (m)", title_font={"size": 14}, row=1, col=1)
fig1.add_trace(
    go.Scatter(
        x=wind_results_baseline[0]["index_values"], y=wind_results_baseline[0]["intensities"], name="baseline wind"
    ),
    row=1,
    col=2,
)
fig1.add_trace(
    go.Scatter(
        x=wind_results_ssp585[0]["index_values"], y=wind_results_ssp585[0]["intensities"], name="wind SSP585 2050"
    ),
    row=1,
    col=2,
)
fig1.update_xaxes(title="Return period (years)", title_font={"size": 14}, row=1, col=2, type="log")
fig1.update_yaxes(title="Max (1 minute) wind speed (m/s)", title_font={"size": 14}, row=1, col=2)

### Accessing the inventory of hazard indicators

Hazard indicator data are 3 dimensional arrays, the dimensions being ('index', 'spatial y', 'spatial x'). In the case of a EPSG:4326 co-ordinate reference system (CRS), for example, this would be ('index', 'latitude', 'longitude'). The data can be in any (CRS); generally the CRS of the original CRS is retained. API requests specify location via latitude and longitude and physrisk performs the necessary conversion.

The 'index' depends on the nature of the hazards. For acute hazards – potential occurrence of events such as flood or tropical cyclone – 'index' will be the return period of the event. For chronic hazards its definition can vary. Chronic hazard indicators may be threshold-based, for example 'mean degree days per year above threshold' or 'mean days per year above temperature threshold'; in these cases 'index' gives the thresholds. Chronic hazards may comprise just a single value (e.g. annual probability of fire) in which case there is just a single element in 'index'.    

The inventory is a list of 'hazard resources' and each resource corresponds to a single array. This means that there are separate entries for the same hazard indicators inferred from different (e.g. CMIP6 or CORDEX) climate models.

For the requests above, the 'path' of the array can be specified explicitly. This is the unique identifier for the hazard resource. But path can also be omitted in which can physrisk applies a set of rules to select a particular array. 

In [6]:
url = base_url + "get_hazard_data_availability"
hazard_resources = requests.post(url, json=request).json()["models"]

In [7]:
def printmd(string):
    display(Markdown(string))


hazard_types = set(r["hazard_type"] for r in hazard_resources)
print(f"Hazards: {hazard_types}")
wind_resources = [r for r in hazard_resources if r["hazard_type"] == "Wind"]
print(f"{len(hazard_resources)} hazard resources in the inventory, of which {len(wind_resources)} are Wind resources.")
iris_model_resource = next(r for r in wind_resources if "iris" in r["path"])
# pp.pprint(iris_model_resource[0])
path = iris_model_resource["path"]
print(f"The resource 'path' is a unique identifier. For the IRIS Wind resource, for example, this is: '{path}'.")
print(
    "Where resources are multi-dimensional arrays (as opposed to an external API), this is also the path of the array."
)
print("The available climate scenarios and years for the IRIS resource are:")
pp.pprint(iris_model_resource["scenarios"])
print("A description is provided in markdown:")
printmd(iris_model_resource["description"])
print("The whole resource:")
pp.pprint(iris_model_resource)

Hazards: {'CoastalInundation', 'Precipitation', 'CombinedInundation', 'RiverineInundation', 'Wind', 'Drought', 'ChronicHeat', 'Hail', 'WaterRisk', 'Fire'}
119 hazard resources in the inventory, of which 2 are Wind hazard resources.
The resource 'path' is a unique identifier. For the IRIS Wind resource, for example, this is: 'wind/iris/v1/max_speed_{scenario}_{year}'.
Where resources are multi-dimensional arrays (as opposed to an external API), this is also the path of the array.
The available climate scenarios and years for the IRIS resource are:
[{'id': 'historical', 'years': [2010]},
 {'id': 'ssp119', 'years': [2050]},
 {'id': 'ssp245', 'years': [2050]},
 {'id': 'ssp585', 'years': [2050]}]
A description is provided in markdown:


Assessing tropical cyclone risk on a global scale given the infrequency of landfalling tropical cyclones and the short period of reliable observations remains a challenge. Synthetic tropical cyclone datasets can help overcome these problems. Here we present a new global dataset created by IRIS, the ImpeRIal college Storm Model. IRIS is novel because, unlike other synthetic TC models, it only simulates the decay from the point of lifetime maximum intensity. This minimises the bias in the dataset. It takes input from 42 years of observed tropical cyclones and creates a 10,000 year synthetic dataset which is then validated against the observations. IRIS captures important statistical characteristics of the observed data. The return periods of the landfall maximum wind speed (1 minute sustained in m/s) are realistic globally. Climate model projections are used to adjust the life-time maximum intensity.
https://www.imperial.ac.uk/grantham/research/climate-science/modelling-tropical-cyclones/


The whole resource:
{'description': 'Assessing tropical cyclone risk on a global scale given the '
                'infrequency of landfalling tropical cyclones and the short '
                'period of reliable observations remains a challenge. '
                'Synthetic tropical cyclone datasets can help overcome these '
                'problems. Here we present a new global dataset created by '
                'IRIS, the ImpeRIal college Storm Model. IRIS is novel '
                'because, unlike other synthetic TC models, it only simulates '
                'the decay from the point of lifetime maximum intensity. This '
                'minimises the bias in the dataset. It takes input from 42 '
                'years of observed tropical cyclones and creates a 10,000 year '
                'synthetic dataset which is then validated against the '
                'observations. IRIS captures important statistical '
                'characteristics of the observed data. The ret

#### Requesting hazard indicators through physrisk directly

As mentioned above, the API is simply using a hosted version of physrisk and it is possible to run the same calculations using physrisk directly, as long as the necessary API keys are present in a credentials.env file.

The requests above can be run in physrisk using a Requester object.

In [ ]:
load_dotenv("../../credentials.env")
# the container is a dependency injection container,
# which allows the calculation to be configured to a particular use-case
container = Container()
# the requester is used to run calculations using the API.
# At this point, we can of course debug into the code and modify as required.
requester = container.requester()
result = requester.get(request_id="get_hazard_data", request_dict=request)